In [ ]:
ipynb_content = {
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "# 🔬 Transfer Learning Comparison Experiment\n",
                "\n",
                "Порівняння різних архітектур та підходів до Transfer Learning"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 1. Імпорт бібліотек"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "import torch\n",
                "import torch.nn as nn\n",
                "import numpy as np\n",
                "import matplotlib.pyplot as plt\n",
                "import seaborn as sns\n",
                "import pandas as pd\n",
                "import json\n",
                "import os\n",
                "import glob\n",
                "from datetime import datetime\n",
                "import warnings\n",
                "warnings.filterwarnings('ignore')\n",
                "\n",
                "# Стилізація графіків\n",
                "plt.style.use('seaborn-v0_8')\n",
                "sns.set_palette(\"husl\")\n",
                "print(\"✅ Бібліотеки успішно імпортовано\")"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 2. Налаштування шляхів"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "# Створення директорій\n",
                "os.makedirs('experiments', exist_ok=True)\n",
                "os.makedirs('results/comparison', exist_ok=True)\n",
                "\n",
                "# Шляхи до результатів\n",
                "RESULTS_DIR = 'results'\n",
                "CHECKPOINTS_DIR = 'checkpoints'\n",
                "\n",
                "print(\"📁 Структура директорій готова\")"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 3. Завантаження результатів експериментів"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "def load_experiment_results():\n",
                "    \"\"\"Завантаження всіх результатів transfer learning\"\"\"\n",
                "    \n",
                "    # Пошук всіх JSON файлів з результатами\n",
                "    result_files = glob.glob(os.path.join(RESULTS_DIR, 'transfer_history_*.json'))\n",
                "    \n",
                "    experiments = []\n",
                "    \n",
                "    for file_path in result_files:\n",
                "        try:\n",
                "            with open(file_path, 'r') as f:\n",
                "                data = json.load(f)\n",
                "                \n",
                "            experiment_info = {\n",
                "                'model': data['model_name'],\n",
                "                'mode': data['mode'],\n",
                "                'best_accuracy': data['best_val_accuracy'],\n",
                "                'timestamp': data['timestamp'],\n",
                "                'history': data['training_history'],\n",
                "                'file_path': file_path\n",
                "            }\n",
                "            experiments.append(experiment_info)\n",
                "            \n",
                "        except Exception as e:\n",
                "            print(f\"Помилка завантаження {file_path}: {e}\")\n",
                "    \n",
                "    print(f\"📊 Завантажено {len(experiments)} експериментів\")\n",
                "    return experiments\n",
                "\n",
                "# Завантаження даних\n",
                "experiments = load_experiment_results()\n",
                "\n",
                "# Перетворення в DataFrame для зручності\n",
                "df_experiments = pd.DataFrame([{\n",
                "    'Model': exp['model'],\n",
                "    'Mode': exp['mode'],\n",
                "    'Best Accuracy': exp['best_accuracy'],\n",
                "    'Epochs': len(exp['history']['train_acc']),\n",
                "    'Timestamp': exp['timestamp']\n",
                "} for exp in experiments])\n",
                "\n",
                "display(df_experiments.sort_values('Best Accuracy', ascending=False))"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 4. Візуалізація порівняння моделей"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "def plot_model_comparison(experiments):\n",
                "    \"\"\"Порівняння точності різних моделей\"\"\"\n",
                "    \n",
                "    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 12))\n",
                "    \n",
                "    # Підготовка даних\n",
                "    models_data = {}\n",
                "    for exp in experiments:\n",
                "        key = f\"{exp['model']}_{exp['mode']}\"\n",
                "        models_data[key] = exp\n",
                "    \n",
                "    # 1. Порівняння найкращої точності\n",
                "    models_names = [f\"{exp['model']}\\n({exp['mode']})\" for exp in experiments]\n",
                "    accuracies = [exp['best_accuracy'] for exp in experiments]\n",
                "    \n",
                "    bars = ax1.bar(models_names, accuracies, color=plt.cm.Set3(np.linspace(0, 1, len(models_names))))\n",
                "    ax1.set_title('🎯 Найкраща точність моделей', fontsize=16, fontweight='bold')\n",
                "    ax1.set_ylabel('Accuracy (%)', fontsize=12)\n",
                "    ax1.tick_params(axis='x', rotation=45)\n",
                "    \n",
                "    # Додавання значень на стовпці\n",
                "    for bar, acc in zip(bars, accuracies):\n",
                "        height = bar.get_height()\n",
                "        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.5,\n",
                "                f'{acc:.2f}%', ha='center', va='bottom', fontweight='bold')\n",
                "    \n",
                "    # 2. Порівняння за типами моделей\n",
                "    model_groups = {}\n",
                "    for exp in experiments:\n",
                "        model = exp['model']\n",
                "        if model not in model_groups:\n",
                "            model_groups[model] = {'feature_extraction': [], 'fine_tuning': []}\n",
                "        model_groups[model][exp['mode']].append(exp['best_accuracy'])\n",
                "    \n",
                "    models = list(model_groups.keys())\n",
                "    x = np.arange(len(models))\n",
                "    width = 0.35\n",
                "    \n",
                "    fe_means = [np.mean(model_groups[m]['feature_extraction']) if model_groups[m]['feature_extraction'] else 0 for m in models]\n",
                "    ft_means = [np.mean(model_groups[m]['fine_tuning']) if model_groups[m]['fine_tuning'] else 0 for m in models]\n",
                "    \n",
                "    ax2.bar(x - width/2, fe_means, width, label='Feature Extraction', alpha=0.8)\n",
                "    ax2.bar(x + width/2, ft_means, width, label='Fine Tuning', alpha=0.8)\n",
                "    \n",
                "    ax2.set_title('📊 Порівняння Feature Extraction vs Fine Tuning', fontsize=16, fontweight='bold')\n",
                "    ax2.set_xlabel('Модель', fontsize=12)\n",
                "    ax2.set_ylabel('Accuracy (%)', fontsize=12)\n",
                "    ax2.set_xticks(x)\n",
                "    ax2.set_xticklabels(models)\n",
                "    ax2.legend()\n",
                "    \n",
                "    # 3. Криві навчання (топ-3 моделі)\n",
                "    top_experiments = sorted(experiments, key=lambda x: x['best_accuracy'], reverse=True)[:3]\n",
                "    \n",
                "    for exp in top_experiments:\n",
                "        label = f\"{exp['model']} ({exp['mode']}) - {exp['best_accuracy']:.2f}%\"\n",
                "        history = exp['history']\n",
                "        epochs = range(1, len(history['val_acc']) + 1)\n",
                "        ax3.plot(epochs, history['val_acc'], 'o-', label=label, linewidth=2, markersize=4)\n",
                "    \n",
                "    ax3.set_title('📈 Криві навчання (топ-3 моделі)', fontsize=16, fontweight='bold')\n",
                "    ax3.set_xlabel('Епоха', fontsize=12)\n",
                "    ax3.set_ylabel('Validation Accuracy (%)', fontsize=12)\n",
                "    ax3.legend()\n",
                "    ax3.grid(True, alpha=0.3)\n",
                "    \n",
                "    # 4. Heatmap порівняння\n",
                "    comparison_data = []\n",
                "    for model in set(exp['model'] for exp in experiments):\n",
                "        for mode in ['feature_extraction', 'fine_tuning']:\n",
                "            accuracies = [exp['best_accuracy'] for exp in experiments \n",
                "                        if exp['model'] == model and exp['mode'] == mode]\n",
                "            if accuracies:\n",
                "                comparison_data.append({'Model': model, 'Mode': mode, 'Accuracy': np.mean(accuracies)})\n",
                "    \n",
                "    if comparison_data:\n",
                "        df_heatmap = pd.DataFrame(comparison_data)\n",
                "        heatmap_data = df_heatmap.pivot(index='Model', columns='Mode', values='Accuracy')\n",
                "        \n",
                "        im = ax4.imshow(heatmap_data.values, cmap='YlGnBu', aspect='auto')\n",
                "        ax4.set_xticks(range(len(heatmap_data.columns)))\n",
                "        ax4.set_yticks(range(len(heatmap_data.index)))\n",
                "        ax4.set_xticklabels(heatmap_data.columns)\n",
                "        ax4.set_yticklabels(heatmap_data.index)\n",
                "        \n",
                "        # Додавання значень у клітинки\n",
                "        for i in range(len(heatmap_data.index)):\n",
                "            for j in range(len(heatmap_data.columns)):\n",
                "                text = ax4.text(j, i, f'{heatmap_data.iloc[i, j]:.2f}%',\n",
                "                           ha=\"center\", va=\"center\", color=\"black\", fontweight='bold')\n",
                "        \n",
                "        ax4.set_title('🌡️ Heatmap порівняння моделей', fontsize=16, fontweight='bold')\n",
                "        plt.colorbar(im, ax=ax4)\n",
                "    \n",
                "    plt.tight_layout()\n",
                "    plt.savefig('results/comparison/model_comparison.png', dpi=300, bbox_inches='tight')\n",
                "    plt.show()\n",
                "\n",
                "# Запуск візуалізації\n",
                "if experiments:\n",
                "    plot_model_comparison(experiments)\n",
                "else:\n",
                "    print(\"❌ Немає даних для візуалізації\")"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 5. Детальний аналіз кривих навчання"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "def plot_detailed_learning_curves(experiments):\n",
                "    \"\"\"Детальні криві навчання для всіх експериментів\"\"\"\n",
                "    \n",
                "    fig, axes = plt.subplots(2, 2, figsize=(20, 12))\n",
                "    axes = axes.flatten()\n",
                "    \n",
                "    # Групування за моделями\n",
                "    models = sorted(set(exp['model'] for exp in experiments))\n",
                "    \n",
                "    for idx, model in enumerate(models):\n",
                "        if idx >= len(axes):\n",
                "            break\n",
                "            \n",
                "        ax = axes[idx]\n",
                "        model_experiments = [exp for exp in experiments if exp['model'] == model]\n",
                "        \n",
                "        for exp in model_experiments:\n",
                "            history = exp['history']\n",
                "            epochs = range(1, len(history['train_acc']) + 1)\n",
                "            \n",
                "            label = f\"{exp['mode']} - {exp['best_accuracy']:.2f}%\"\n",
                "            \n",
                "            # Train/Val accuracy\n",
                "            ax.plot(epochs, history['train_acc'], '--', alpha=0.7, \n",
                "                   label=f'Train ({exp[\\\"mode\\\"]})')\n",
                "            ax.plot(epochs, history['val_acc'], '-', linewidth=2,\n",
                "                   label=f'Val {label}')\n",
                "        \n",
                "        ax.set_title(f'{model.upper()} - Криві навчання', fontsize=14, fontweight='bold')\n",
                "        ax.set_xlabel('Епоха')\n",
                "        ax.set_ylabel('Accuracy (%)')\n",
                "        ax.legend()\n",
                "        ax.grid(True, alpha=0.3)\n",
                "    \n",
                "    # Learning rates\n",
                "    if len(experiments) > 0:\n",
                "        ax = axes[3]\n",
                "        for exp in experiments[:4]:  # Перші 4 експерименти\n",
                "            history = exp['history']\n",
                "            if 'learning_rates' in history:\n",
                "                epochs = range(1, len(history['learning_rates']) + 1)\n",
                "                label = f\"{exp['model']} ({exp['mode']})\"\n",
                "                ax.plot(epochs, history['learning_rates'], 'o-', label=label, markersize=4)\n",
                "        \n",
                "        ax.set_title('📉 Learning Rate по епохах', fontsize=14, fontweight='bold')\n",
                "        ax.set_xlabel('Епоха')\n",
                "        ax.set_ylabel('Learning Rate')\n",
                "        ax.set_yscale('log')\n",
                "        ax.legend()\n",
                "        ax.grid(True, alpha=0.3)\n",
                "    \n",
                "    plt.tight_layout()\n",
                "    plt.savefig('results/comparison/detailed_learning_curves.png', dpi=300, bbox_inches='tight')\n",
                "    plt.show()\n",
                "\n",
                "if experiments:\n",
                "    plot_detailed_learning_curves(experiments)"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 6. Статистичний аналіз"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "def perform_statistical_analysis(experiments):\n",
                "    \"\"\"Статистичний аналіз результатів\"\"\"\n",
                "    \n",
                "    print(\"📊 СТАТИСТИЧНИЙ АНАЛІЗ РЕЗУЛЬТАТІВ\")\n",
                "    print(\"=\" * 50)\n",
                "    \n",
                "    if not experiments:\n",
                "        print(\"Немає даних для аналізу\")\n",
                "        return\n",
                "    \n",
                "    # Основні статистики\n",
                "    accuracies = [exp['best_accuracy'] for exp in experiments]\n",
                "    \n",
                "    print(f\"Загальна кількість експериментів: {len(experiments)}\")\n",
                "    print(f\"Середня точність: {np.mean(accuracies):.2f}%\")\n",
                "    print(f\"Максимальна точність: {np.max(accuracies):.2f}%\")\n",
                "    print(f\"Мінімальна точність: {np.min(accuracies):.2f}%\")\n",
                "    print(f\"Стандартне відхилення: {np.std(accuracies):.2f}%\")\n",
                "    \n",
                "    print(\"\\n\" + \"=\" * 50)\n",
                "    \n",
                "    # Порівняння Feature Extraction vs Fine Tuning\n",
                "    fe_acc = [exp['best_accuracy'] for exp in experiments if exp['mode'] == 'feature_extraction']\n",
                "    ft_acc = [exp['best_accuracy'] for exp in experiments if exp['mode'] == 'fine_tuning']\n",
                "    \n",
                "    if fe_acc and ft_acc:\n",
                "        print(\"\\n🔍 ПОРІВНЯННЯ РЕЖИМІВ:\")\n",
                "        print(f\"Feature Extraction: {len(fe_acc)} експ., середня {np.mean(fe_acc):.2f}%\")\n",
                "        print(f\"Fine Tuning: {len(ft_acc)} експ., середня {np.mean(ft_acc):.2f}%\")\n",
                "        \n",
                "        if np.mean(ft_acc) > np.mean(fe_acc):\n",
                "            improvement = ((np.mean(ft_acc) - np.mean(fe_acc)) / np.mean(fe_acc)) * 100\n",
                "            print(f\"🎯 Fine Tuning кращий на {improvement:.1f}%\")\n",
                "        else:\n",
                "            improvement = ((np.mean(fe_acc) - np.mean(ft_acc)) / np.mean(ft_acc)) * 100\n",
                "            print(f\"🎯 Feature Extraction кращий на {improvement:.1f}%\")\n",
                "    \n",
                "    # Топ моделі\n",
                "    print(\"\\n🏆 ТОП-3 МОДЕЛІ:\")\n",
                "    top_models = sorted(experiments, key=lambda x: x['best_accuracy'], reverse=True)[:3]\n",
                "    \n",
                "    for i, exp in enumerate(top_models, 1):\n",
                "        print(f\"{i}. {exp['model']} ({exp['mode']}): {exp['best_accuracy']:.2f}%\")\n",
                "\n",
                "perform_statistical_analysis(experiments)"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 7. Рекомендації та висновки"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "def generate_recommendations(experiments):\n",
                "    \"\"\"Генерація рекомендацій на основі результатів\"\"\"\n",
                "    \n",
                "    print(\"💡 РЕКОМЕНДАЦІЇ ТА ВИСНОВКИ\")\n",
                "    print(\"=\" * 60)\n",
                "    \n",
                "    if not experiments:\n",
                "        print(\"Недостатньо даних для формування рекомендацій\")\n",
                "        return\n",
                "    \n",
                "    # Знаходимо найкращі підходи\n",
                "    best_overall = max(experiments, key=lambda x: x['best_accuracy'])\n",
                "    \n",
                "    fe_experiments = [exp for exp in experiments if exp['mode'] == 'feature_extraction']\n",
                "    ft_experiments = [exp for exp in experiments if exp['mode'] == 'fine_tuning']\n",
                "    \n",
                "    best_fe = max(fe_experiments, key=lambda x: x['best_accuracy']) if fe_experiments else None\n",
                "    best_ft = max(ft_experiments, key=lambda x: x['best_accuracy']) if ft_experiments else None\n",
                "    \n",
                "    print(\"\\n🎯 НАЙКРАЩІ РЕЗУЛЬТАТИ:\")\n",
                "    print(f\"Загальний чемпіон: {best_overall['model']} ({best_overall['mode']}) - {best_overall['best_accuracy']:.2f}%\")\n",
                "    \n",
                "    if best_fe:\n",
                "        print(f\"Найкращий Feature Extraction: {best_fe['model']} - {best_fe['best_accuracy']:.2f}%\")\n",
                "    if best_ft:\n",
                "        print(f\"Найкращий Fine Tuning: {best_ft['model']} - {best_ft['best_accuracy']:.2f}%\")\n",
                "    \n",
                "    print(\"\\n📋 РЕКОМЕНДАЦІЇ:\")\n",
                "    \n",
                "    # Рекомендації на основі результатів\n",
                "    if best_fe and best_ft:\n",
                "        accuracy_gap = best_ft['best_accuracy'] - best_fe['best_accuracy']\n",
                "        \n",
                "        if accuracy_gap > 5:\n",
                "            print(\"✅ Fine Tuning значно кращий. Використовуйте fine-tuning для максимальної точності\")\n",
                "        elif accuracy_gap > 2:\n",
                "            print(\"✅ Fine Tuning трохи кращий. Рекомендується fine-tuning\")\n",
                "        else:\n",
                "            print(\"⚡ Різниця незначна. Feature Extraction може бути ефективнішим через швидкість\")\n",
                "    \n",
                "    # Рекомендації по моделям\n",
                "    model_performance = {}\n",
                "    for exp in experiments:\n",
                "        if exp['model'] not in model_performance:\n",
                "            model_performance[exp['model']] = []\n",
                "        model_performance[exp['model']].append(exp['best_accuracy'])\n",
                "    \n",
                "    print(\"\\n🏗️ РЕКОМЕНДАЦІЇ ПО МОДЕЛЯМ:\")\n",
                "    for model, accs in model_performance.items():\n",
                "        avg_acc = np.mean(accs)\n",
                "        print(f\"  {model}: середня точність {avg_acc:.2f}% ({len(accs)} експ.)\")\n",
                "    \n",
                "    print(\"\\n🚀 ПРАКТИЧНІ ПОРАДИ:\")\n",
                "    print(\"1. Для продакшену: використовуйте найкращу модель з fine-tuning\")\n",
                "    print(\"2. Для прототипування: feature extraction - швидше та простіше\")\n",
                "    print(\"3. Експериментуйте з learning rates для кожної моделі\")\n",
                "    print(\"4. Використовуйте early stopping для уникнення перенавчання\")\n",
                "\n",
                "generate_recommendations(experiments)"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "## 8. Збереження звіту"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": null,
            "metadata": {},
            "outputs": [],
            "source": [
                "def save_comparison_report(experiments):\n",
                "    \"\"\"Збереження повного звіту\"\"\"\n",
                "    \n",
                "    if not experiments:\n",
                "        print(\"❌ Немає даних для звіту\")\n",
                "        return\n",
                "    \n",
                "    timestamp = datetime.now().strftime(\"%Y%m%d_%H%M%S\")\n",
                "    report_path = f'results/comparison/transfer_comparison_report_{timestamp}.txt'\n",
                "    \n",
                "    with open(report_path, 'w', encoding='utf-8') as f:\n",
                "        f.write(\"=\" * 70 + \"\\n\")\n",
                "        f.write(\"          ЗВІТ ПОРІВНЯННЯ TRANSFER LEARNING МОДЕЛЕЙ\\n\")\n",
                "        f.write(\"=\" * 70 + \"\\n\\n\")\n",
                "        \n",
                "        f.write(f\"Дата генерації: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n\")\n",
                "        f.write(f\"Кількість експериментів: {len(experiments)}\\n\\n\")\n",
        "        \n",
                "        # Топ результати\n",
                "        f.write(\"ТОП-5 ЕКСПЕРИМЕНТІВ:\\n\")\n",
                "        f.write(\"-\" * 50 + \"\\n\")\n",
                "        \n",
                "        top_experiments = sorted(experiments, key=lambda x: x['best_accuracy'], reverse=True)[:5]\n",
                "        \n",
                "        for i, exp in enumerate(top_experiments, 1):\n",
                "            f.write(f\"{i:2d}. {exp['model']:15} | {exp['mode']:18} | {exp['best_accuracy']:6.2f}% | \"\n",
                "                   f\"{len(exp['history']['train_acc']):2d} епох\\n\")\n",
                "        \n",
                "        # Статистика\n",
                "        accuracies = [exp['best_accuracy'] for exp in experiments]\n",
                "        \n",
                "        f.write(f\"\\nСТАТИСТИКА:\\n\")\n",
                "        f.write(f\"  Середня точність: {np.mean(accuracies):.2f}%\\n\")\n",
                "        f.write(f\"  Максимальна точність: {np.max(accuracies):.2f}%\\n\")\n",
                "        f.write(f\"  Мінімальна точність: {np.min(accuracies):.2f}%\\n\")\n",
                "        \n",
                "        # Рекомендації\n",
                "        best_overall = max(experiments, key=lambda x: x['best_accuracy'])\n",
                "        \n",
                "        f.write(f\"\\nРЕКОМЕНДАЦІЇ:\\n\")\n",
                "        f.write(f\"  Найкраща модель: {best_overall['model']} ({best_overall['mode']})\\n\")\n",
                "        f.write(f\"  Точність: {best_overall['best_accuracy']:.2f}%\\n\")\n",
                "        f.write(f\"  Епох навчання: {len(best_overall['history']['train_acc'])}\\n\")\n",
                "    \n",
                "    print(f\"✅ Звіт збережено: {report_path}\")\n",
                "\n",
                "save_comparison_report(experiments)\n",
                "\n",
                "print(\"\\n🎉 АНАЛІЗ ЗАВЕРШЕНО!\")\n",
                "print(\"📁 Результати збережено в папці results/comparison/\")\n",
                "print(\"📊 Графіки та звіти готові для презентації\")"
            ]
        }
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "Python 3",
            "language": "python",
            "name": "python3"
        },
        "language_info": {
            "codemirror_mode": {
                "name": "ipython",
                "version": 3
            },
            "file_extension": ".py",
            "mimetype": "text/x-python",
            "name": "python",
            "nbconvert_exporter": "python",
            "pygments_lexer": "ipython3",
            "version": "3.8.0"
        }
    },
    "nbformat": 4,
    "nbformat_minor": 4
}

# Зберігаємо як .ipynb файл
import json

with open('experiments/transfer_comparison.ipynb', 'w', encoding='utf-8') as f:
    json.dump(ipynb_content, f, indent=2, ensure_ascii=False)

print("✅ Файл experiments/transfer_comparison.ipynb успішно створено!")

NameError: name 'null' is not defined